<table style="width:100%">
  <tr style="background-color:#FFFFFF">
    <td style="width:10%">
        <img src="https://play-lh.googleusercontent.com/K9FC23HoIYy_idt3LL26zrzMchOGORVzLNmi3L5u-MaghdjpdgHAWv7wQlscNmo9wYjC=s180-rw" alt="DiDi" width=120></td>
    <td style="width:10%">
        <img src="https://logodownload.org/wp-content/uploads/2019/05/99-app-logo.png" alt="99" width=120></td>
      <td>
          <h1 style="text-align:left; text-indent :2em">DATA TRANSFORMATION • PART 3</h1>
          <h2 style="text-align:left; text-indent :2.4em;">ISIS SANTOS COSTA • 2020</h2>
      </td>
  </tr>
</table>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Resuming: retrieving the DFs

In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('df_orders_t3.csv') as f: cols = f.readline().split(',')
del cols[0]
cols[-1] = cols[-1].replace('\n','')
df_orders = pd.read_csv('df_orders_t3.csv', usecols=cols) # reading previously transformed data
df_orders.head(2)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,pickup_month,pickup_weekinmonth
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD,2014-04-11,Friday,15,0,18,1,4,2
1,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD,2014-04-12,Saturday,15,1,0,0,4,2


In [4]:
with open('df_trips_t3.csv') as f: cols = f.readline().split(',')
del cols[0]
cols[-1] = cols[-1].replace('\n','')
df_trips = pd.read_csv('df_trips_t3.csv', usecols=cols) # reading previously transformed data
df_trips.head(2)

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_type,dropoff_longitude,dropoff_latitude,payment_type,...,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,duration,duration_minutes,duration_type,pickup_month,pickup_weekinmonth
0,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,Standard,-73.995022,40.765441,CRD,...,Saturday,21,1,9,1,0 days 00:10:23,10.383333,10-20min,5,4
1,14054197,2014-04-10 15:14:12,2014-04-10 15:49:05,12.4,-73.991044,40.749704,Standard,-73.909173,40.823500,CSH,...,Thursday,15,0,15,1,0 days 00:34:53,34.883333,30-40min,4,2


## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Combining the tables: outer join 🔗

⏳ <i> Considering the time to load/save the DFs, and that DF trips has same information as DF orders for the converted trips, let's join them into one

### Listing information added when orders get converted: trips_features

In [5]:
df_orders.columns

Index(['order_id', 'passenger_id', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'payment_type', 'pickup_date', 'pickup_dayofweek',
       'pickup_week', 'pickup_isweekend', 'pickup_hour', 'pickup_isrush',
       'pickup_month', 'pickup_weekinmonth'],
      dtype='object')

In [6]:
df_trips.columns

Index(['order_id', 'pickup_datetime', 'dropoff_datetime', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'rate_type', 'dropoff_longitude',
       'dropoff_latitude', 'payment_type', 'fare_amount', 'tip_amount',
       'total_amount', 'passenger_id', 'pickup_date', 'pickup_dayofweek',
       'pickup_week', 'pickup_isweekend', 'pickup_hour', 'pickup_isrush',
       'duration', 'duration_minutes', 'duration_type', 'pickup_month',
       'pickup_weekinmonth'],
      dtype='object')

In [7]:
# columns to add for orders that became trips
trips_features = set(df_trips.columns) - set(df_orders.columns)
trips_features

{'dropoff_datetime',
 'dropoff_latitude',
 'dropoff_longitude',
 'duration',
 'duration_minutes',
 'duration_type',
 'fare_amount',
 'rate_type',
 'tip_amount',
 'total_amount',
 'trip_distance'}

### Adding trips_features to the table of all orders

In [8]:
trips_features.add('order_id')

In [9]:
df_orders_tF = pd.merge(df_orders, df_trips[trips_features], on='order_id', how='outer')

In [10]:
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,...,fare_amount,trip_distance,dropoff_longitude,duration,dropoff_datetime,duration_minutes,dropoff_latitude,total_amount,duration_type,tip_amount
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD,2014-04-11,Friday,15,0,...,9.5,1.6,-74.000814,0 days 00:11:34,2014-04-11 18:18:19,11.566667,40.718173,13.2,10-20min,2.2
1,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD,2014-04-12,Saturday,15,1,...,22.0,5.5,-73.991395,0 days 00:26:51,2014-04-12 00:47:19,26.850000,40.758526,27.0,20-30min,4.0
2,6433699,5234567814730471,2014-04-11 23:51:28,-73.987125,40.747428,CRD,2014-04-11,Friday,15,0,...,8.0,1.6,-74.005073,0 days 00:08:13,2014-04-11 23:59:41,8.216667,40.740963,10.8,5-10min,1.8


🔎 <i> Checking if trips features are the same as in df_trips, for same order_id:

In [11]:
df_trips.loc[df_trips['order_id']==6433697]

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_type,dropoff_longitude,dropoff_latitude,payment_type,...,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,duration,duration_minutes,duration_type,pickup_month,pickup_weekinmonth
4218327,6433697,2014-04-11 18:06:45,2014-04-11 18:18:19,1.6,-74.015684,40.714476,Standard,-74.000814,40.718173,CRD,...,Friday,15,0,18,1,0 days 00:11:34,11.566667,10-20min,4,2


<i> Nice, looks good! ✔️

### 🗃️ Reordering the columns of the new DF df_orders_tF (<i>t</i>ransformed, <i>F</i>inal)

👀 <i> Let's organize the columns, allocating related features adjacent to each other, making it easier to compare them. </i> 👀

In [12]:
cols = list(df_orders_tF.columns.values)
cols

['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'payment_type',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_week',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_month',
 'pickup_weekinmonth',
 'rate_type',
 'fare_amount',
 'trip_distance',
 'dropoff_longitude',
 'duration',
 'dropoff_datetime',
 'duration_minutes',
 'dropoff_latitude',
 'total_amount',
 'duration_type',
 'tip_amount']

In [13]:
cols = ['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_month',
 'pickup_week',
 'pickup_weekinmonth',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_latitude',
 'pickup_longitude',
 'dropoff_latitude',
 'dropoff_longitude',
 'dropoff_datetime',
 'duration',
 'duration_minutes',
 'duration_type',
 'trip_distance',
 'total_amount',
 'fare_amount',
 'tip_amount',
 'rate_type',
 'payment_type']
cols

['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_month',
 'pickup_week',
 'pickup_weekinmonth',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_latitude',
 'pickup_longitude',
 'dropoff_latitude',
 'dropoff_longitude',
 'dropoff_datetime',
 'duration',
 'duration_minutes',
 'duration_type',
 'trip_distance',
 'total_amount',
 'fare_amount',
 'tip_amount',
 'rate_type',
 'payment_type']

In [14]:
df_orders_tF = df_orders_tF[cols]
df_orders_tF.head()

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,...,2014-04-12 09:24:44,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,...,2014-04-12 19:21:31,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD


<b>(</b>  Θ_Θ<b>)</b> <i>  &nbsp;&nbsp;&nbsp; Nice </i>&nbsp;&nbsp;&nbsp;<b>(</b>  Θ_Θ<b>)</b>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Feature engineering: converting 📍coordinates into 🗺️neighborhoods

### Preparing for heavy processing: exporting the dataframe as transformed so far 📤

<i> In case the Kernel... leaves us </i>  &nbsp;&nbsp; • ‖ •  &nbsp;&nbsp; [ Kernel ] 🎒😎➡️👋

In [23]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 12min 39s


### Preparing for heavy processing: autotime + tqdm (progress bar)

In [1]:
#!pip install ipython-autotime
%load_ext autotime

time: 0 ns


In [2]:
import tqdm
from tqdm.notebook import tqdm_notebook

time: 3.99 s


### Importing: Geopandas • Geopy (Nominatim+RateLimiter) • PyPlot • Plotly_express

In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

time: 51 s


In [4]:
#!pip install plotly_express==0.4.0
import plotly_express as px

time: 40.5 s


In [5]:
#!pip install geopy
import geopy
from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter

time: 2.11 s


In [6]:
#!pip install wheel
#!pip install pipwin

#!pipwin install numpy
#!pipwin install pandas
#!pipwin install shapely
#!pipwin install gdal
#!pipwin install fiona
#!pipwin install pyproj
#!pipwin install six
#!pipwin install rtree
#!pipwin install geopandas

time: 0 ns


In [7]:
#!pip install geopandas # <<== This approach did't work, had to run the cells above
#!conda install -c conda-forge geopandas # <<== This approach did't work, had to run the cells above
import geopandas as gpd

time: 21.8 s


### Constructing Geocoder

In [8]:
locator = Nominatim (user_agent='myGeocoder', timeout=24000) # extended timeout

time: 0 ns


### Reverse geocoding • Pilot test ✔️

📝 <i> It took some testing to figure out the administrative organization of the city and to find out how to return results in spite of missing data.
    <p> <b>Regarding missing data</b> </i>🕳️<i>: not all the addresses in the reference presented information for all of the queried fields, and when a missing value was reached, the whole fetching was being interrupted. The solution was to fetch the entire address - a dictionary - and then <u>filter</u> the needed data from 'address' in the DF, applying the method <u>setdefault</u> to the dictionaries.</p>
    <p> <b>Regarding the division of the city</b> </i>🗺️<i>: the initial intent to dismember the city into neighborhoods led to choosing <u>'neighbourhood'</u> from the geocoder address. It happened that on one side it presented a lot of missing data and, on the other side, there seemed to be too many unique values: dividing into too many districts wouldn't ease the analysis any much. Its bigger container - <u>'suburb'</u> - has been thus added to the study.

In [28]:
df_orders_geo = df_orders_tF[['order_id', 'pickup_latitude', 'pickup_longitude']][0:12]

df_orders_geo['address'] = df_orders_geo.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)

df_orders_geo['neighborhood'] = df_orders_geo['address']
df_orders_geo['suburb'] = df_orders_geo['address']

for i in range(len(df_orders_geo)):
    df_orders_geo['address'][i].setdefault('neighbourhood')
    df_orders_geo['neighborhood'][i] = df_orders_geo['address'][i]['neighbourhood']
    
    df_orders_geo['address'][i].setdefault('suburb')
    df_orders_geo['suburb'][i] = df_orders_geo['address'][i]['suburb']
    
df_orders_geo.head(3)

,order_id,pickup_latitude,pickup_longitude,address,neighborhood,suburb
0,6433697,40.714476,-74.015684,"{'house_number': '4', 'road': 'Vesey Street', ...",Battery Park City,Manhattan
1,6433698,40.713247,-73.958403,"{'house_number': '175', 'road': 'Roebling Stre...",Williamsburg,Brooklyn
2,6433699,40.747428,-73.987125,"{'house_number': '25', 'road': 'West 31st Stre...",Koreatown,Manhattan


time: 8.61 s


In [29]:
100 * df_orders_geo['neighborhood'].value_counts() / len(df_orders_geo)

Midtown East          16.666667
Battery Park City      8.333333
Carnegie Hill          8.333333
Financial District     8.333333
Williamsburg           8.333333
Koreatown              8.333333
Lenox Hill             8.333333
Greenwich Village      8.333333
Yorkville              8.333333
Midtown South          8.333333
Name: neighborhood, dtype: float64

time: 0 ns


In [30]:
100 * df_orders_geo['suburb'].value_counts() / len(df_orders_geo)

Manhattan    83.333333
Brooklyn     16.666667
Name: suburb, dtype: float64

time: 1.44 s


In [31]:
100 * df_orders_geo.loc[df_orders_geo['suburb']=='Manhattan', 'neighborhood'].value_counts() / len(df_orders_geo)

Midtown East          16.666667
Battery Park City      8.333333
Carnegie Hill          8.333333
Financial District     8.333333
Koreatown              8.333333
Lenox Hill             8.333333
Greenwich Village      8.333333
Yorkville              8.333333
Midtown South          8.333333
Name: neighborhood, dtype: float64

time: 438 ms


### Reverse geocoding • Full scale 📈: 1<sup>st</sup> trial with Nominatim, by list ❌

<i> The following code has been tried earlier, but kept getting interrupted before completion: </i>

In [ ]:
# rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.0002)

In [ ]:
# tqdm_notebook.pandas()

# df_orders['neighborhood'] = df_orders['pickup_coordinates'].progress_apply(rgeocode)
# df_orders.head(2)'''

### Reverse geocoding • Full scale 📈: n<sup>th</sup> trial with Nominatim, by list ❌

<i> After many trials (and some Kernels lost down the road), decided the following would be the last. <br>
    On hitting a missing value, the whole processing for the other rows has been lost (msg in the collapsed cells). </i>

In [32]:
df_orders_tF['pickup_address'] = df_orders_tF.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)

TypeError: `address` must not be None

time: 4h 9min 3s


In [33]:
df_orders_tF['pickup_address'].head()

KeyError: 'pickup_address'

time: 2.31 s


<i> First reaction: 'Yaaay! Just four hours \o/' </i>😃<i> <br> 
    After checking with '.head()' </i>😒
    <p> <i> Hm... So, fetching was interrupted when a missing value was reached. </i>

### ( Resuming after Kernel shut down • Retrieving data: nice it'd been saved! 😅 )

In [9]:
import pandas as pd

time: 0 ns


In [10]:
import warnings
warnings.filterwarnings('ignore')

time: 0 ns


In [11]:
df_orders_tF = pd.read_csv('df_orders_tF.csv', index_col=0) # reading previously transformed data
df_orders_tF.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.2,9.5,2.2,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.0,22.0,4.0,Standard,CRD


time: 8min 41s


### Reverse geocoding • Full scale 📈 [ try / except ] ❌

<i> The following approach adds an 'antidote' for interrupting the query when is to define a 'ignore Exception', the beautiful pair try / except:

In [12]:
try:
    df_orders_tF['pickup_address'] = df_orders_tF.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)
except Exception:
    pass

time: 3h 55min 10s


<i> Again, in case the Kernel is gone:

In [13]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 43min 5s


In [14]:
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.2,9.5,2.2,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.0,22.0,4.0,Standard,CRD
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.8,8.0,1.8,Standard,CRD


time: 18.7 s


In [15]:
df_orders_tF['pickup_address'].head(3)

KeyError: 'pickup_address'

time: 1min 32s


In [16]:
df_orders_tF.shape

(15000000, 25)

time: 203 ms


<i> Learning how the combo [ try / expect + apply ] operate, the painful way... Ok, this leads us to the next trial: item by item. </i>

### Reverse geocoding • Nominatim [ try / except ] querying item by item ✔️ (kinda... 90 days!)

In [48]:
df_orders_tF['address'] = [''] * len(df_orders_tF)

for i in range(len(df_orders_tF)):
    coordinates = str(df_orders_tF['pickup_latitude'][i]) + ',' + str(df_orders_tF['pickup_longitude'][i])
    try:
        df_orders_tF['address'][i] = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass

KeyboardInterrupt: 

time: 1d 7h 22s


<i> And, once again: </i>

In [94]:
df_orders_tF.to_csv('df_orders_tF_218k.csv')

time: 36min 7s


In [49]:
df_orders_tF.head()

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ..."
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre..."
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD,"{'house_number': '25', 'road': 'West 31st Stre..."
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,...,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD,"{'building': 'Fulton Street', 'road': 'Fulton ..."
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,...,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD,"{'amenity': 'Rite Aid', 'house_number': '1849'..."


time: 1.53 s


<i> First results look good!
    <p> Let's check if all the 15,000,000 addresses have been retrieved: </i></p>

In [51]:
df_orders_tF['address'].count()

15000000

time: 1.03 s


In [52]:
i

218209

time: 0 ns


<b><i> Uh oh! Only 218,210 results have been fetched... In 31.5 hours! It means...  <u>90 days</u> for the whole data. Uh oh... </i>

#### 90 x 🕛🕐🕑🕒🕓🕔🕕🕖🕗🕘🕙🕚🕛🕐🕑🕒🕓🕔🕕🕖🕗🕘🕙🕚 <br>

### Reverse geocoding • Nominatim [ numba / njit / numpy vectorize ] ✔️ (kinda: not fast enough)

In [81]:
! pip install numba

time: 51.3 sRequirement already satisfied: numba in c:\users\graca\anaconda3\lib\site-packages (0.51.2)



In [82]:
from numba import njit
import numpy as np

time: 39.7 s


In [83]:
@njit(parallel=True)
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF_sm = df_orders_tF[i:i+100]
df_orders_tF_sm['address'] = get_address(df_orders_tF_sm['pickup_latitude'].values, df_orders_tF_sm['pickup_longitude'].values)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'locator':[0m [1m[1mcannot determine Numba type of <class 'geopy.geocoders.nominatim.Nominatim'>[0m
[1m
File "<ipython-input-83-bf488ac5f1bf>", line 5:[0m
[1mdef get_address(lat, lng):
    <source elided>
    try:
[1m        address = (locator.reverse(coordinates)).raw['address']
[0m        [1m^[0m[0m
[0m

time: 14.6 s


In [84]:
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF_sm = df_orders_tF[i:i+100]
df_orders_tF_sm['address'] = np.vectorize(get_address)(df_orders_tF_sm['pickup_latitude'], df_orders_tF_sm['pickup_longitude'])

time: 52.2 s


In [87]:
df_orders_tF_sm.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
218209,6651906,5234567812372677,2014-04-14 01:46:47,4,16,2,2014-04-14,Monday,0,1,...,0 days 00:05:40,5.666667,5-10min,1.1,8.0,6.0,1.0,Standard,CRD,"{'building': 'Symphony House', 'road': 'West 5..."
218210,6651907,5234567816149281,2014-04-14 03:32:11,4,16,2,2014-04-14,Monday,0,3,...,0 days 00:03:05,3.083333,<=5min,0.6,6.6,4.5,1.1,Standard,CRD,"{'house_number': '54', 'road': 'West Houston S..."


time: 31 ms


In [88]:
df_orders_tF_sm.tail(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
218307,6652004,5234567813368025,2014-04-13 23:44:36,4,15,2,2014-04-13,Sunday,1,23,...,0 days 00:14:12,14.200000,10-20min,6.4,21.5,20.5,0.0,Standard,CSH,"{'house_number': '601', 'road': '10th Avenue',..."
218308,6652005,5234567812392667,2014-04-13 18:18:48,4,15,2,2014-04-13,Sunday,1,18,...,0 days 00:15:31,15.516667,10-20min,1.0,11.0,10.5,0.0,Standard,CSH,"{'house_number': '645', 'road': '2nd Avenue', ..."


time: 31 ms


In [90]:
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF['address'] = np.vectorize(get_address)(df_orders_tF['pickup_latitude'], df_orders_tF['pickup_longitude'])

KeyboardInterrupt: 

time: 1h 24min 47s


### Reverse geocoding • Taking Einstein's advice

#### <i> « As simple as possible... </i>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Einstein__as_simple_as_possible.jpg'>

#### <i> ... but not simpler » </i>

<img src='img/Simplicity-Through-Thoughtful-Reduction.png'>

#### Defining a function for a « good enough » location of neighborhood

In [ ]:
def get_address(lat,lng):
    if long < x:
        address = 'Staten Island'
    elif 2:
        address = 'Bronxs'
    elif 2:
        address = 'Queens'
    elif 2:
        address = 'Brooklyn'
    else:
        address = 'Manhattan'
        

## Exporting results for retrieval

In [ ]:
df_orders_tF.to_csv('df_orders_tF.csv')

<i>Note: This Notebook (Transformation) would entail the whole transforming process, but as some instabilities may interrupt the Kernel requiring a restart from zero, it has been split for a smoothier process.</i>

# Next: [EDA](99__EDA.ipynb)